#Lab.12 / IBM3202 – Protein folding *ab initio* using Rosetta

#Theoretical aspects.
Why can we perform structure predictions for a given protein sequence?

As we already saw in the homology modelling tutorial there is a growing need to solve the 3D structure of proteins. As evolution has evolved from a finite number of families of proteins there are usually closely related proteins that has been already solved and can be used in strategies as the aforementioned homology modelling. But there is also a chance of you encountering a protein with no closely related homologs which has been already structurally solved, in this last case scenario we can use modelling derived from first principles or more commonly refered as *Ab initio* modelling, which is template agnostic.



As we have discussed during the course, there are many ways you can predict a protein’s structure by knowing only its sequence: homology-based comparative modelling, determination of coevolutionary couplings using a myriad of homologous protein sequences and ab initio structure prediction. Ab initio comes from the Latin and means “from first principles”, in which the idea is to arrive at a protein structure just by knowing the principles governing protein folding.

Where do these principles come from? As we have seen in our lectures, the ever-increasing number of solved protein structures by different methods (X-ray, NMR, cryoEM) has enabled to define that, regardless their primary sequence, proteins do not explore all possible configurations. This very simple observation enabled further use of this information to perform statistical analysis of the different conformations that a protein sequences usually explore: typical amino acid environments and distances between amino acids in any given protein, secondary structure predictions based on sequence information alone, angle and distance geometries between secondary structure elements, dihedral distributions of backbones and side chains, etc. Then, this information can be used to define energy functions which enable us to determine which structures are more probable for a given sequence (an inverse folding problem).


While this method has been used for several protein structure predictions, the exponential growth of the conformational space of a protein that can be explored by a given protein sequence with respect to its protein length severely limits its use to proteins with fewer than 100 residues. The rationale behind such exact number is that a protein domain, i.e. considered as a region (sequence) of a protein that can fold independently onto a given structure by its own, typically has a maximum in its typical length distribution at ~100 residues for most proteins (**Fig 1**). Thus, there seems to be an optimal number of residues that allow proteins to form a hydrophobic core big enough to stably fold.

<figure>
<center>
<img src="https://ars.els-cdn.com/content/image/1-s2.0-S1359027898000042-gr3.gif"/>

<figcaption>FIGURE 1. Length distribution of proteins domains. The arrow represents the maximum of the distribution, which in turn represents the minimum of a folding energy function that depends solely on the entropic cost of packing a n-polymer, the hydration cost for its radios of gyration, and the average internal enthalpy of contact formation.  <br> Adapted from Xu & Nussinov (1998)<i> Folding & Design.</i></figcaption></center>
</figure>

Although this is used as a general rule, there are plenty of exceptions as shown in the above distribution, of domains that form well below or beyond the optimal 100 ± 40 residues. Some extremely short proteins can adopt stable structures in solution (i.e. the β-stable hairpin we studied through MD, or the villin head we are about to fold), and immensely large proteins that can form a huge hydrophobic core. This noisy distribution highlights that there is not a unique solution to the protein folding problem, and the principles governing the folding of ideal, regular-sized globular proteins may not be the same than that of outlier, small domains. Today, we will put that to the test.

#General Overview

In this tutorial, we will employ Rosetta to predict the protein structure of the 35-residue headpiece of villin from G. gallus. Although compiling may be messy, and the documentation is awfully dispersed and obscure through tens of different versions and small unannounced modifications, Rosetta is by far the most successful engine in de novo design and prediction of proteins structures (Kaufmann *et al.* (2010) *Biochemistry*).

Briefly, we will perform the following tasks:

Part I – Prepare the fragment libraries required for protein structure prediction using Rosetta

Part II – Execute Rosetta to generate 10 ab initio models

Part III – Analyze the results using py3Dmol and Rosetta

Part IV – Prepare a larger number of models (>10,000) using two different fragment libraries

##Folding protocol overview


As stated in the **AbinitioRelax [documentation](https://)**

> The AbinitioRelax application consists of two main steps.
1. The first step is a coarse-grained fragment-based search through conformational space using a knowledge-based "centroid" score function that favors protein-like features (Abinitio).
2. The second optional step is all-atom refinement using the Rosetta full-atom forcefield (Relax). The "Relax" step is considerably more compute-intensive and time-consuming than the first step.

There is also commented the fact that:

> For increased conformational sampling, this application is easily parallelized by executing numerous jobs each using a unique random number seed (see Options section below). This is typically done by submitting multiple jobs to a computer cluster or distributed grid. Since the full-atom energy function is very sensitive to imperfect atomic interactions and more noise will exist with insufficient sampling, convergence towards the native structure may require a significant amount of sampling. Additionally, to increase your chance of sampling the correct topology, a diverse set of homologous sequences, preferably with sequence changes that may have a greater impact on sampling like deletions and differences in conserved positions, may also be run since a homologue may converge towards the native structure with significantly less sampling (See Bradley *et al.* (2005). Toward high-resolution de novo structure prediction for small proteins. *Science*, 309(5742), 1868-1871).






## Folding.py
This script performs fragment insertion for an input protein sequence. The
sequence may be explicit, in a FASTA file, or in a PDB file. Two fragment files,
preferably one longer than the other, must also be provided. Fragment insertion
is accompanied by a Monte Carlo assessment allowing the conformation to escape
local minima. Output structures from this protocol are intended to proceed into
a refinement step (such as that in refinement.py) to produce reasonable
estimates of the protein conformation.

Instructions:

1. ensure that your PDB file is in the current directory
2. run the script:
  - from commandline  
      - > python D060_Folding.py
  - from within python/ipython              
      - run D060_Folding.py

Author: Evan H. Baugh
    revised and motivated by Robert Schleif

Updated by Boon Uranukul, 6/9/12
Simplified special constant seed initialization ~ Labonte

References:
    P. Bradley, K. Misura, and D. Baker, "Toward high-resolution de novo
        structure prediction for small proteins", *Science* 309 (5742)
        1868-1871 (2005).

"""

## The complete folding protocol, steps included in the "sample_folding" definition



This sample script is setup for usage with
    commandline arguments,
    default running within a python interpreter,
    or for import within a python interpreter,
        (exposing the sample_folding method)

The method sample_folding:
1.  creates a pose from the desired sequence (fullatom)
2.  set all of the pose's backbone torsion angles
3.  creates a (fullatom) reference copy of the pose
4.  creates Movers for switching between fullatom and centroid
5.  convert both poses to centroid
6.  creates a MoveMap with all backbone torsion angles free
7.  sets up ClassicFragmentMovers for inserting fragments backbone structures for long and short fragments
8.  creates RepeatMovers for the long and short fragment Movers
9.  create a PyMOL_Observer for viewing intermediate output
10. creates low and high resolution ScoreFunctions
11. sets up a RepeatMover on a TrialMover of a SequenceMover setup the TrialMover
      
      -  create a SequenceMover with the:
          - RepeatMover on the long fragment Mover
          - RepeatMoversepeatMover on the short fragment Mover

      -  create a MonteCarlo object for assessing moves
      -  create the TrialMover (on the SequenceMover)
      -  create the RepeatMover (on the TrialMover)
12. creates a (Py)JobDistributor for managing multiple trajectories
13. stores the original score evaluation (centroid)
14. performs low-resolution folding:
  - set necessary variables
      - reload the starting (centroid) pose
      - change the pose's PDBInfo.name, for the PyMOLMover
      - reset the MonteCarlo object (for the TrialMover)
  - perform sampling and assessment using the final RepeatMover
  - convert the lowest scoring decoy to fullatom using the SequenceMover
  - export the (lowest scoring) decoy structure
      - recover the lowest scoring (centroid) decoy
      - store the decoy score
      - convert the decoy to fullatom
      - guess potential disulfide bridges
      - output the decoy structure using the PyJobDistributor
      - export the decoy structure using the PyMOL_Observer
15. outputs the score evaluations

The second method, guess_disulfides, is called in sample_folding to print out
cysteine residues which are close to each other in the decoy structure.


#Part I – Prepare the fragment libraries required for protein structure prediction using Rosetta

The sequence of the protein for which we want to predict its structure, the **G34L mutant** of the headpiece subdomain of the **Villin** protein from *G. gallus*, is the following:
> MLSDEDFKAFGMTRSAFANLPLWKQQNLKKEKLLF

The first thing we need to do is to generate a fragment library that we will require for modelling our protein. As you remember from the lectures, these libraries are essential for closing loops between segments and for determining the most probable local structure for a given sequence.

To generate these libraries, we recommend using **Robetta (http://old.robetta.org/)**, a webserver that is kept by the same laboratory in charge of maintaining Rosetta. Here, one typically will go onto the “Submit” section of the Fragment Library service, and then input the name of the target protein in the “Target Name” field and its full sequence in Fasta format in the “Paste Fasta” field. Finally, we would normally press “Submit” to start the fragment library generation.

Depending on the protein sequence length, these fragment library generations can take up to several hours. Therefore, we are providing these files for your uninterrupted work (0- fragment_libraries/villin folder). If you take a quick look onto these files, you should see:

* 3-residue long fragment file (i.e. aat000_03...)

* 9-residue long fragment file (i.e. aat000_09...)

* 3 temporal files (i.e. *.dat, *.check, *.checkpoint)

* a Fasta file with your sequence (i.e. *.fasta)

* 3 secondary structure prediction files (i.e. *.jufo_ss, *.psipred, *.psipred_ss2)

* a database file (i.e. *.rdb)

We are only interested in the fragment files. Open one of them in a text editor to examine its content. For each, 200 fragments will be generated, and the content is as follows:


* Column -- Meaning
* 1 -- blank

* 2-5 -- PDB code for the fragment origin

* 7 -- chain ID for the origin PDB

* 9-13 -- PDB residue number for the origin PDB

* 15 -- amino acid identity in the origin PDB

* 17 -- secondary structure for the origin PDB (Helix, Loop, Extended/beta) 19-27 -- phi

* 28-36 -- psi

* 37-45 -- omega

* 46-54 -- C-alpha x coordinate for origin PDB (optional)

* 55-63 -- C-alpha y coordinate for origin PDB (optional)

* 65-73 -- C-alpha z coordinate for origin PDB (optional)

* 74-79 -- unknown (unused)

* 80-85 -- unknown (unused)

* 86 -- Literal "P" (unused)

* 87-89 -- fragment position number, pose numbered (unused)

* 91 -- Literal "F"(unused)

* 92-94 -- fragment number (unused)

**QUESTION!!**
* Why are the position coordinate of the fragments optional?
* Why we do not require any information about the structural coordinates of the rest of the atoms?

# Part II – Execute Rosetta to generate 10 *ab initio* models

Once we have generated the fragment libraries, we will employ them along with Rosetta. In its core, Rosetta is quite similar to Gromacs in that it consists of a series of individual applications or codes that share in common a language and libraries. In this case, we will use the application named **AbinitioRelax**. To examine this application in detail, please Google the name of the application followed by Rosetta, and you’ll immediately find the documentation page alongside with useful references. **Do it. Seriously. Rosetta is an extremely convoluted and complicated program!**

In [ ]:
# Notebook setup
import sys
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()

Here we copy test files for running the method

In [ ]:
#Here we download the input files from Github
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/input_files/input_files_ai.zip
#Here, we unzip the downloaded files
!unzip input_files_ai.zip

Here we create the outs folder needed for saving the results from folding

In [ ]:
!mkdir outs

Following is the protocol for running from colab

### Definition of method for folding protocol

In [ ]:
import optparse    # for sorting options

import pyrosetta
import pyrosetta.rosetta as rosetta

from pyrosetta import (
    init, pose_from_sequence, pose_from_file, Pose, MoveMap, create_score_function, get_fa_scorefxn,
    MonteCarlo, TrialMover, SwitchResidueTypeSetMover, PyJobDistributor,
)
from pyrosetta.rosetta import core, protocols
init(extra_options = "-scorefile_format text")

In [ ]:
def sample_folding(sequence,
        long_frag_filename, long_frag_length,
        short_frag_filename, short_frag_length,
        kT = 3.0, long_inserts = 1, short_inserts = 3, cycles = 40,
        jobs = 1, job_output = 'fold_output'):
    """

    """
    # 1. create a pose from the desired sequence (fullatom)
    # the method pose_from_sequence produces a complete IDEALIZED
    #    protein conformation of the input sequence, the ResidueTypeSet (second
    #    argument below) may be varied, and this method supports non-proteogenic
    #    chemistry (though it is still a Rosetta Residue). however this syntax
    #    is more involved and not robust to user errors, and not presented here
    # small differences in bond lengths and bond angles WILL change the results,
    #### if you desire an alternate starting conformation, alter steps
    ####     1. and 2. as you please
    pose = pose_from_sequence(sequence, 'fa_standard')

    # 2. linearize the pose by setting backbone torsions to large values
    # the method make_pose_from_sequence does not create the new pose's
    #    PDBInfo object, so its done here, without it an error occurs later
    pose.pdb_info(rosetta.core.pose.PDBInfo( pose.total_residue() ))
    for i in range(1, pose.total_residue() + 1):
        pose.set_omega(i, 180)
        pose.set_phi(i, -150)    # reasonably straight
        pose.set_psi(i, 150)
        #### if you want to see the decoy scores, the PDBInfo needs these lines
        #pose.pdb_info().chain(i, 'A')    # necessary to color by score
        #pose.pdb_info().number(i, i)    # for PDB numbering
    ####

    # 3. create a (fullatom) reference copy of the pose
    test_pose = Pose()
    test_pose.assign( pose )
    test_pose.pdb_info().name('linearized pose')

    # 4. create centroid <--> fullatom conversion Movers
    to_centroid = SwitchResidueTypeSetMover('centroid')
    # centroid Residue objects, of amino acids, have all their sidechain atoms
    #    replaced by a single representative "atom" to speed up calculations
    to_fullatom = SwitchResidueTypeSetMover('fa_standard')

    # 5. convert the poses to centroid
    to_centroid.apply(pose)
    to_centroid.apply(test_pose)

    # 6. create the MoveMap, all backbone torsions free
    movemap = MoveMap()
    movemap.set_bb(True)
    # minimizing the centroid chi angles (the sidechain centroid atoms) is
    #    almost always USELESS since this compression is performed for speed,
    #    not accuracy and clashes usually occur when converting to fullatom

    # 7. setup the ClassicFragmentMovers
    # for the long fragments file
    # this "try--except" is used to catch improper fragment files
    try:
        fragset_long = core.fragment.ConstantLengthFragSet( long_frag_length , long_frag_filename )
        #### the ConstantLengthFragSet is overloaded, this same
        ####    ConstantLengthFragSet can be obtained with different syntax
        # to obtain custom fragments, see Generating Fragment Files below
    except:
        raise IOError('Make sure long_frag_length matches the fragments in\n\
            long_frag_file and that long_frag_file is valid')
    long_frag_mover = protocols.simple_moves.ClassicFragmentMover(fragset_long, movemap)
    # and for the short fragments file
    # this "try--except" is used to catch improper fragment files
    try:
        fragset_short = core.fragment.ConstantLengthFragSet(short_frag_length, short_frag_filename)
    except:
        raise IOError('Make sure short_frag_length matches the fragments in\n\
            short_frag_file and that short_frag_file is valid')
    short_frag_mover = protocols.simple_moves.ClassicFragmentMover(fragset_short, movemap)

    # 8. setup RepeatMovers for the ClassicFragmentMovers
    insert_long_frag = protocols.moves.RepeatMover(long_frag_mover, long_inserts)
    insert_short_frag = protocols.moves.RepeatMover(short_frag_mover, short_inserts)

    # 9. create a PyMOL_Observer for exporting structures to PyMOL (optional)
    # the PyMOL_Observer object owns a PyMOLMover and monitors pose objects for
    #    structural changes, when changes are detected the new structure is
    #    sent to PyMOL
    # fortunately, this allows investigation of full protocols since
    #    intermediate changes are displayed, it also eliminates the need to
    #    manually apply the PyMOLMover during a custom protocol
    # unfortunately, this can make the output difficult to interpret (since you
    #    aren't explicitly telling it when to export) and can significantly slow
    #    down protocols since many structures are output (PyMOL can also slow
    #    down if too many structures are provided and a fast machine may
    #    generate structures too quickly for PyMOL to read, the
    #    "Buffer clean up" message
    # uncomment the line below to use PyMOL_Observer
    ##    AddPyMOLObserver(test_pose, True)

    # 10. create ScoreFunctions
    # for low-resolution, centroid, poses necessary for the TrialMover's
    #    MonteCarlo object (see below)
    scorefxn_low = create_score_function('score3')
    # for high-resolution, fullatom, poses necessary for scoring final output
    #    from the PyJobDistributor (see below)
    scorefxn_high = get_fa_scorefxn() #  create_score_function('standard', 'score12')

    # 11. setup a RepeatMover on a TrialMover of a SequenceMover
    # -setup a TrialMover
    #    a. create a SequenceMover of the fragment insertions
    #### add any other moves you desire
    folding_mover = protocols.moves.SequenceMover()
    folding_mover.add_mover(insert_long_frag)
    folding_mover.add_mover(insert_short_frag)
    #    b. create a MonteCarlo object to define success/failure
    # must reset the MonteCarlo object for each trajectory!
    mc = MonteCarlo(test_pose, scorefxn_low, kT)
    # c. create the TrialMover
    trial = TrialMover(folding_mover, mc)

    #### for each trajectory, try cycles number of applications

    # -create the RepeatMover
    folding = protocols.moves.RepeatMover(trial, cycles)

    # 12. create a (Py)JobDistributor
    jd = PyJobDistributor(job_output, jobs, scorefxn_high)

    # 13. store the score evaluations for output
    # printing the scores as they are produced would be difficult to read,
    #    Rosetta produces a lot of verbose output when running
    scores = [0]*(jobs + 1)
    scores[0] = scorefxn_low(pose)

    # 14. perform folding by
    counter = 0    # for exporting to PyMOL
    while not jd.job_complete:
        # a. set necessary variables for the new trajectory
        # -reload the starting pose
        test_pose.assign(pose)
        # -change the pose's PDBInfo.name, for the PyMOL_Observer
        counter += 1
        test_pose.pdb_info().name(job_output + '_' + str(counter))
        # -reset the MonteCarlo object (sets lowest_score to that of test_pose)
        mc.reset(test_pose)

        #### if you create a custom protocol, you may have additional
        ####    variables to reset, such as kT

        #### if you create a custom protocol, this section will most likely
        ####    change, many protocols exist as single Movers or can be
        ####    chained together in a sequence (see above) so you need
        ####    only apply the final Mover
        # b. apply the refinement protocol
        folding.apply(test_pose)

        ####
        # c. export the lowest scoring decoy structure for this trajectory
        # -recover the lowest scoring decoy structure
        mc.recover_low(test_pose)
        # -store the final score for this trajectory
        scores[counter] = scorefxn_low(test_pose)
        # -convert the decoy to fullatom
        # the sidechain conformations will all be default,
        #    normally, the decoys would NOT be converted to fullatom before
        #    writing them to PDB (since a large number of trajectories would
        #    be considered and their fullatom score are unnecessary)
        # here the fullatom mode is reproduced to make the output easier to
        #    understand and manipulate, PyRosetta can load in PDB files of
        #    centroid structures, however you must convert to fullatom for
        #    nearly any other application
        to_fullatom.apply(test_pose)
        # -guess what cysteines are involved in disulfide bridges
        guess_disulfides(test_pose)
        # -output the fullatom decoy structure into a PDB file
        jd.output_decoy(test_pose)
        # -export the final structure to PyMOL
        test_pose.pdb_info().name(job_output + '_' + str(counter) + '_fa')

        #### if you want to see the decoy scores, uncomment the line below
        #scorefxn_high( test_pose )

    # 15. output the score evaluations
    print( '===== Centroid Scores =====' )
    print( 'Original Score\t:\t', scores[0] )
    for i in range(1, len(scores)):    # print out the job scores
        # the "[:14].ljust(14)" is to force the text alignment
        print( (job_output + '_' + str( i ))[:14].ljust(14) +\
            '\t:\t', scores[i] )

    return scores    # for other protocols

# locates all cysteines, make bonds between them, output the bonds that lower
#    the score
def guess_disulfides(pose, cutoff = 6.0):
    """
    A quick method for probing a protein for cysteine residues close to each
        other (within  <cutoff>  )
    """
    # find all cysteine residues and consider possible disulfides
    cys = [i for i in range(1, pose.total_residue() + 1)
        if pose.residue(i).name1() == 'C']
    partners = [0]*sum( range(len(cys)) )    # all disulfides possible
    i = 0
    # create all combinations
    for first in range(len(cys[:-1])):
        for second in cys[first + 1:]:
            partners[i] = (cys[first], second)
            i += 1
    # try each disulfide, if it lowers the score, print it to screen
    print( '='*80 )
    print( 'Potential Disulfides:' )
    for pair in partners:
        # for a fullatom cysteine in PyRosetta, the 6th atom is sulfur
        separation = (pose.residue(pair[0]).xyz(6) -
            pose.residue(pair[1]).xyz(6)).norm
        if separation < cutoff:
            print( 'between (pose numbered) residue' , pair[0] , 'and' ,\
                pair[1] , '|' , separation , 'Angstrom separation' )
    print( '='*80 )
    # to manipulate disulfide bonds, use:
    # formation:    form_disulfide(pose.conformation(), 6, 16)
    # cleavage:     change_cys_state(6, 'CYS', pose.conformation() )
    #               change_cys_state(16, 'CYS', pose.conformation() )



## Here we define input variables


-   The long_frag_length MUST match long_frag_filename (9 in this example)         

-   The short_frag_length MUST match short_frag_filename (3 here)
-   100 trajectories is low, sampling protein conformations requires many trials, typically hundreds (800-1000) trajectories are attempted         
-   The "temperature" parameter, kT, is set to 1.0, a neutral value, larger kT increase the diversity of sampling (easily escape local minima) but requires compensation with more trajectories smaller kT decreases the chance of sampling "useless" space but cannot easily escape local minima (the default kT = 3.0)
- There are no common values for long_inserts, short_inserts, or cycles more inserts indicates greater sampling with fragments while greater cycles produces more sampling in general (though in this case only fragment insertion is performed) however, only one selection (Metropolis Criteria) is applied per cycle, thus a proposed structure with too many fragments inserted is likely to be rejected but additional trials increases the computation time noticeably here, as is typical is Rosetta, a fairly conservative change is proposed (1 9-mer and 3 3-mers) per cycle with many cycles

In [ ]:
# the user may input a PDB file, fasta file, or sequence directly
# PDB file option
pose = Pose()
# Fasta file option
fasta_filename = './t000_.fasta'
if fasta_filename:    # defaults to off, empty string
    f = open(fasta_filename, 'r')    # open the file
    sequence = f.readlines()    # read the text
    f.close()    # close it
    # removing the trailing "\n" and any header lines
    sequence = [line.strip() for line in sequence if not '>' in line]
    sequence = ''.join( sequence )    # combine into a single sequence
elif options.sequence:
    sequence = options.sequence
#else:
 #   pose_from_file(pose, pdb_filename)
 #   sequence = pose.sequence()
#Checks for the sequence in a fasta, then direct, and finally from a PDB file.  If no PDB file is given, it will load the default.

# fragment files options
long_frag_filename = './aat000_09_05.200_v1_3'
long_frag_length = 9
short_frag_filename = './aat000_03_05.200_v1_3'
short_frag_length = 3
# folding protocol options, see above
kT = 3.0
long_inserts = 1
short_inserts = 3
cycles = 300
# PyJobDistributor options
jobs = 10
job_output = './outs/fold_output'                  #Prefix

## Here we finally call the folding function

In [ ]:
sample_folding(sequence,
    long_frag_filename, long_frag_length,
    short_frag_filename, short_frag_length,
    kT, long_inserts, short_inserts, cycles,
    jobs, job_output)

# Part III – Analyze the results

As a result of running Rosetta, two things will happen:
1. All the 10 predicted structures for villin headpiece will be outputted as PDB files
2. An energy score and statistics file of the output, called score.fsc, will also be continually generated

In [ ]:
!sudo apt install jq

In [ ]:
#As we obtain a Json scorefile we use the jq bash json parser to filter the decoy name and total_score
#in order to get the lowest energy model
!jq '.decoy , .total_score' /content/outs/fold_output.fasc

Now is your turn to the 2PPZ(native.pdb) pdb and align it to the structure with the predicted structure with the lowest energy using py3Dmol and BioPython!!

In [ ]:
!pip install biopython
#We import Bio.PDB module again
from Bio.PDB import *
parser = PDBParser()

In [ ]:
#The following code was created by Anders Steen Christensen
#from the University of Basel and is available at
#https://gist.github.com/andersx/6354971

import Bio.PDB

# Select what residues numbers you wish to align
# and put them in a list
start_id = 1
end_id   = 35
atoms_to_be_aligned = range(start_id, end_id + 1)

# Start the parser
pdb_parser = Bio.PDB.PDBParser(QUIET = True)

# Get the structures
ref_structure = pdb_parser.get_structure("reference", "/content/native.pdb")
sample_structure = pdb_parser.get_structure("sample", "/content/outs/fold_output_4.pdb")

# Use the first model in the pdb-files for alignment
# Change the number 0 if you want to align to another structure
ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Make a list of the atoms (in the structures) you wish to align.
# In this case we use CA atoms whose index is in the specified range
ref_atoms = []
sample_atoms = []

# Iterate of all chains in the model in order to find all residues
for ref_chain in ref_model:
  # Iterate of all residues in each model in order to find proper atoms
  for ref_res in ref_chain:
    # Check if residue number ( .get_id() ) is in the list
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Append CA atom to list
      ref_atoms.append(ref_res['CA'])

# Do the same for the sample structure
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Now we initiate the superimposer:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Print RMSD:
print('The calculated RMSD is:')
print (super_imposer.rms)

# Save the aligned version of one of the chains of 6ANE
io = Bio.PDB.PDBIO()
io.set_structure(sample_structure)
io.save("Your_best_model_aligned.pdb")

In [ ]:
#Here, write down the code to install py3Dmol
!pip install py3Dmol

In [ ]:
#And we already learned how to import these modules, right?
import py3Dmol
view=py3Dmol.view()
view.addModel(open('/content/Your_best_model_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('/content/native.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{'cartoon': {'color':'yellow'}})
view.setStyle({'chain':'B'},{'cartoon': {'color':'green'}})
view.show()

# Appendix A. Going deeper

## A Real Example
"""
All of the default variables and parameters used above are specific to
the example with "test_in.pdb", which is supposed to be simple,
straightforward, and speedy. Here is a more practical example:

Pheromone ER-23 is a small protein (51 amino acids) with 5 disulfide bridges.
The formation of these bonds may significantly effect folding. Suppose you
are interested in the significance of these disulfide bonds as folding
constraints and want to predict the protein without them using PyRosetta.

1. Obtain the protein sequence of RCSB PDB 1HA8 (instructions above)
        (for the example here, download the PDB file and ensure it is clean)
2. Make two fragment files using the "Generate Fragment Files"
        (instructions above)
        -of 9-mers (the long fragments)
        -of 3-mers (the short fragments)
3. Make a directory containing:
        -the PDB file for 1HA8 (cleaned of HETATMs and waters)
            lets name it "1HA8.clean.pdb" here
        -the 9-mer fragment file for 1HA8
            lets name it "1HA8.frag9" here
        -the 3-mer fragment file for 1HA8
            lets name it "1HA8.frag3" here
        -this sample script (technically not required, but otherwise the
            commands in 4. would change since folding.py would not be here)
4. Run the script from the commandline with appropriate arguments:

>python folding.py --pdb_filename 1HA8.clean.pdb --long_frag_filename 1HA8.frag9 --long_frag_length 9 --short_frag_filename 1HA8.frag3 --short_frag_length 3 --jobs 100 --job_output 1HA8_folding_output --kT 1.0 --long_inserts 1 --short_inserts 3 --cycles 200 --disulfides 1

        -The long_frag_length MUST match long_frag_filename (9 in this example)
        -The short_frag_length MUST match short_frag_filename (3 here)
        -100 trajectories is low, sampling protein conformations requires many
            trials, typically hundreds (800-1000) trajectories are attempted
        -The "temperature" parameter, kT, is set to 1.0, a neutral value,
            larger kT increase the diversity of sampling (easily escape local
            minima) but requires compensation with more trajectories
            smaller kT decreases the chance of sampling "useless" space but
            cannot easily escape local minima (the default kT = 3.0
        -There are no common values for long_inserts, short_inserts, or cycles
            more inserts indicates greater sampling with fragments while
            greater cycles produces more sampling in general (though in this
            case only fragment insertion is performed)
            however, only one selection (Metropolis Criteria) is applied per
            cycle, thus a proposed structure with too many fragments inserted
            is likely to be rejected but additional trials increases the
            computation time noticeably
            here, as is typical is Rosetta, a fairly conservative change is
            proposed (1 9-mer and 3 3-mers) per cycle with many cycles

5. Wait for output, this will take a while (performing 100 trajectories
        of loop modeling involving 1*3*200 total moves per trajectory)
6. Analyze the results (see INTERPRETING RESULTS above)

Note: this is NOT intended to be used for realistic centroid folding,
it merely provides a "skeleton" for the code in PyRosetta. It may be useful
for preliminary investigation but the best protocols are somewhat
protein-specific, there is no current universal folding method. As mentioned
above, this script only touches the low-resolution aspect of Rosetta folding
and must be accompanied by a high-resolution step to yield complete results.
Generally, a protocol similar to the one presented here with more drastic
sampling and a larger number of trials should be sufficient to find realistic
folds (after refinement).

"""




## Changing Folding Sampling
"""
Rosetta is a structure prediction tool and all predictions rely on the
techniques (moves) used to generate proposed structures. There is NO GENERAL
SOLUTION for predicting a protein fold from scratch and fragment insertion
is an effective way of sampling the relevant space. Since fragments are
sequence and secondary structure (prediction) dependent, the resultant folds
are biochemically feasible and in accordance with our prior observations about
protein stucture (the PDB).

For speed, Rosetta folding (and the folding algorithm presented here) is
performed in low-resolution (centroid) mode. This method of folding is
abstractly looking for structures which obey certain size constraints (i.e. the
rg score term) but do not unrealistically clash (i.e. the vdw score term) that
are achieved easily (i.e. fragment insertion, a fast algorithm). Sufficient
sampling and downstream refinement will convert these general folds (the output
of this script) into reasonable estimates of the native protein structure.

Many other sampling techniques and methods are available. PyRosetta was designed
to make novel investigation of these ideas easy. Recommend literature include
topics in protein structure prediction, Monte Carlo Markov Chain processes,
Rosetta protocols, and simulated annealing algorithms. When developing custom
protocols, you MUST understand the Movers available in PyRosetta (see the other
sample scripts, specifically refinement.py). Unfortunately, you cannot build
novel Mover classes in PyRosetta. Fortunately, it is easy to implement similar\
code on your own once you understand how moves and selection are performed.

Please try alternate sampling methods to better understand how these
algorithms perform and to find what moves best suite your problem.

"""




## Changing Folding Scoring
"""
The Rosetta "force-field" is a biasing function (see MCMC algorithms) used to
predict protein structures based on what is observed about other protein
structures. These scoring methods supply the constraints necessary to bias
structure prediction but incorporate a LARGE number of fine tuned parameters.
There is no easy way to optimize all relevant constants and performance has been
increased by balancing speed and accuracy.

Many other relevant metrics exist for protein structures and discriminating
decoys (incorrect predictions) from actual proteins is an ongoing task. Please
experiment with different ScoreTypes in PyRosetta, their weights, and the
combination of terms. Unfortunately, you cannot build novel ScoreTypes (scoring
methods) in PyRosetta. Fortunately, it is easy to implement similar code on your
own once you understand how scoring biases MCMC simulations.

Please try alternate scoring functions or unique selection methods to better
understand which scoring terms contribute to performance and to find what
scoring best suites your problem.

"""